Config extractor of obfuscated umbral stealer samples and CLEANED (de4dot) samples  writen in DotNet. 
hashes: 
4ec777ca1708d332e9f7184507430caed00dfbc3f87334275ac0c0631ad007b7
dc50156f82253ecf0d2cbcbfad5f5cec41c0d56dce7754dc920966cec548112c

In [ ]:

import clr  # Import pythonnet's Common Language Runtime interface
file_path = "path_to_your_exe"  # Replace with the actual path to your .NET assembly
clr.AddReference("dnlib")  # Add reference to dnlib.dll (but this needs to point to a real path or GAC-loaded assembly)

import dnlib  # Attempting to import dnlib as a Python module 
from System.Reflection import Assembly, BindingFlags, MethodInfo
from dnlib.DotNet import ModuleDef, ModuleDefMD
from dnlib.DotNet.Emit import OpCodes
from dnlib.DotNet.Writer import ModuleWriterOptions
from dnlib.DotNet import *  # Import everything from dnlib.DotNet namespace
from dnlib.DotNet.Emit import OpCodes  # Import IL OpCodes used to analyze method instructions

module = dnlib.DotNet.ModuleDefMD.Load(file_path)  # Load .NET assembly into dnlib


In [ ]:
import base64
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from base64 import b64decode

def decrypt_data(encrypted_data, key, iv):
    # Decode Base64 inputs
    encrypted_bytes = b64decode(encrypted_data)
    key_bytes = b64decode(key)
    iv_bytes = b64decode(iv)

    # Split into ciphertext and authentication tag
    ciphertext = encrypted_bytes[:-16]
    auth_tag = encrypted_bytes[-16:]

    # Decrypt using AES-GCM
    aesgcm = AESGCM(key_bytes)
    plaintext = aesgcm.decrypt(iv_bytes, ciphertext + auth_tag, None)
    return plaintext.decode('utf-8')


In [ ]:
# This for obfuscated umbral stealer samples
for type in module.GetTypes():
    for method in type.Methods:
        if not method.HasBody:
            continue

        instrs = method.Body.Instructions

        for i in range (len(instrs)):
            instr = instrs[i]
            if instrs[i].OpCode == OpCodes.Stloc_3 and instrs[i-1].OpCode == OpCodes.Ldstr :
                enc_mutex = instrs[i-1].Operand
                enc_version = instrs[i-3].Operand
                enc_webhook = instrs[i-4].Operand
                enc_iv = instrs[i-6].Operand
                enc_key = instrs[i-8].Operand


webhook = decrypt_data(enc_webhook, enc_key, enc_iv)
version = decrypt_data(enc_version, enc_key, enc_iv)
mutex = decrypt_data(enc_mutex, enc_key, enc_iv)
print("C2: ", webhook)
print("Version: ", version)
print("Mutex: ", mutex)

            
    

In [ ]:
# This for cleaned umbral stealer samples
for type in module.GetTypes():
    for method in type.Methods:
        if not method.HasBody:
            continue

        instrs = method.Body.Instructions

        for i in range (len(instrs)):
            instr = instrs[i]
            if instr.OpCode == OpCodes.Call:
                target = instr.Operand
            # print(instr)
                if instr.OpCode == OpCodes.Call or instr.OpCode == OpCodes.Callvirt:
                    operand = instr.Operand
                    # print(operand)
                    if operand.Name == "FromBase64String":
                        if instrs[i-1].OpCode == OpCodes.Stloc_3:
                            enc_mutex = instrs[i-2].Operand
                            enc_version = instrs[i-4].Operand
                            enc_webhook = instrs[i-5].Operand
                            enc_iv = instrs[i-7].Operand
                            enc_key = instrs[i-9].Operand


webhook = decrypt_data(enc_webhook, enc_key, enc_iv)
version = decrypt_data(enc_version, enc_key, enc_iv)
mutex = decrypt_data(enc_mutex, enc_key, enc_iv)
print("C2: ", webhook)
print("Version: ", version)
print("Mutex: ", mutex)
    